In [14]:
### generate complex data 
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1):

    # Generate a random input signal
    k = len(true_coefficients)
    m, n = true_coefficients[0].shape


    x = np.random.randn(n_samples, m) + 1j*np.random.randn(n_samples, m)

    # Generate the desired signal by passing the input through the known system
    y = np.zeros((n_samples, n), dtype=np.complex128)

    ### y is complex data
    for j in range(k):
        y += np.dot(np.roll(x, j, axis=0), true_coefficients[j])

    # Add noise
    # noise = noise_level * np.random.randn(n_samples, n) + 1j*noise_level * np.random.randn(n_samples, n)
    # y += noise
    
    return x, y

# Example usage
n_samples = 30000
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2

k = 1
m = 3
n = 2
## coef is complex 
true_coefficients = [np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]

noise_level = 0.05
x, y = generate_data(n_samples, true_coefficients, noise_level)
print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[-0.35882895-0.84681752j  0.6034716 -0.55395797j -1.66478853-0.12986603j]
 [-0.70017904+0.19904257j  1.15139101-1.17771595j  1.85733101+0.34118233j]
 [-1.51117956+0.09682765j  0.64484751-0.04215581j -0.98060789+0.9331908j ]]
Desired signal: [[ 0.24995128+0.72692461j -3.39696997+0.58424488j]
 [ 3.43472766-3.98540717j  2.66208791+0.44149349j]
 [ 2.00783119+2.63443481j -0.92408249+2.29807431j]]


In [15]:
### using kalman filter to estimate the coefficients
def kalman_filter(x, y, k):
    n_samples = len(y)
    m = x.shape[1]
    n = y.shape[1]
    
    # Initialize the state estimate
    # Initialize w: k x m x n randomly
    w = np.ones((k, m, n), dtype=np.complex128) * 0.1

    # PQR matrices, initialized as identity matrices, each one is k x n x n
    P = np.array([np.eye(m*n) for _ in range(k)]) * 1
    # Q = np.array([np.eye(m*n) for _ in range(k)]) * 0.01
    R = np.array([np.eye(n) for _ in range(k)]) * 1
    

    for i in range(k, n_samples):
        # Predict
        for j in range(k):
            ## w[j] add some samll noise
            # w[j] = w[j] +  np.random.randn(m, n) * 0.01
            # P[j] = P[j] 
            pass 
        
        # Update
        
        ### complex number
        y_hat = np.zeros(n,dtype=np.complex128)
        for j in range(k):
            y_hat += np.dot(x[i-j], w[j])
        
        
        for j in range(k):
            ## w[j] add K[j] * e
            W_temp = w[j].T.reshape(-1,1) 
            H_temp = np.zeros((n, m*n), dtype=np.complex128)
            for l in range(n):
                H_temp[l, m*l:m*(l+1)] = x[i-j] 
            S = np.dot(H_temp, np.dot(P[j], np.conj(H_temp.T)) ) + R[j]
            K = np.dot(P[j], np.dot(np.conj(H_temp.T), np.linalg.inv(S))) 

            e = y[i] - y_hat
            W_temp += np.dot(K.reshape(m*n, n), e.reshape(-1,1)) 
            w[j] = W_temp.reshape(n, m).T

            P[j] = np.dot((np.eye(m*n) - np.dot(K, H_temp)), P[j]) ### P is reasonable 
            
        # if i % 5 == 0:
        #     print(w[0])
            
    return w

In [16]:
## Example usage

estimated_coefficients = kalman_filter(x, y, k)
print("True coefficients:", true_coefficients)
print("Estimated coefficients:", estimated_coefficients)

C:\Users\mioshi\AppData\Local\Temp\ipykernel_22896\3379472039.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
  P[j] = np.dot((np.eye(m*n) - np.dot(K, H_temp)), P[j]) ### P is reasonable


True coefficients: [array([[ 0.44122749-0.9092324j , -0.33087015-0.59163666j],
       [ 2.43077119+0.18760323j, -0.25209213-0.32986996j],
       [ 0.10960984-1.19276461j,  1.58248112-0.20487651j]])]
Estimated coefficients: [[[ 0.44103571-0.9090808j  -0.33091231-0.59187824j]
  [ 2.43084303+0.18750058j -0.25201513-0.32975644j]
  [ 0.10983692-1.19250398j  1.58305712-0.20465447j]]]
